In [85]:
import pandas as pd

In [130]:
import numpy as np

In [86]:
from time_series_generator import TimeseriesGenerator

In [87]:
import tensorflow as tf

In [88]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [89]:
dff = pd.read_csv("data/Main_Demo.csv")

In [90]:
dff.head

<bound method NDFrame.head of       Unnamed: 0    Bill MerchandiseID                             Product  \
0              0  532657         21314       Small glass heart trinket pot   
1              1  563214         22383              Lunch bag suki design    
2              2  507597         22561         Wooden school colouring set   
3              3  491634         21588      Retro spot giant  tube matches   
4              4  496007        85232B    Set/3 russian doll stacking tins   
...          ...     ...           ...                                 ...   
1926        2463  530843         22764  Rustic wooden cabinet, glass doors   
1927        2464  513975         22321            Bird decoration red spot   
1928        2465  580541         23166      Medium ceramic top storage jar   
1929        2466  544754         85187             S/12 mini rabbit easter   
1930        2468  558700         22557    Plasters in tin vintage paisley    

      Quota    BillDate  Amount  

In [91]:
sales_rd = dff.copy(deep=True)

In [92]:
sales_rd.head

<bound method NDFrame.head of       Unnamed: 0    Bill MerchandiseID                             Product  \
0              0  532657         21314       Small glass heart trinket pot   
1              1  563214         22383              Lunch bag suki design    
2              2  507597         22561         Wooden school colouring set   
3              3  491634         21588      Retro spot giant  tube matches   
4              4  496007        85232B    Set/3 russian doll stacking tins   
...          ...     ...           ...                                 ...   
1926        2463  530843         22764  Rustic wooden cabinet, glass doors   
1927        2464  513975         22321            Bird decoration red spot   
1928        2465  580541         23166      Medium ceramic top storage jar   
1929        2466  544754         85187             S/12 mini rabbit easter   
1930        2468  558700         22557    Plasters in tin vintage paisley    

      Quota    BillDate  Amount  

In [93]:
sales_rd = sales_rd.dropna()

In [94]:
cols = ['Bill',	'MerchandiseID',	'Product',	'Quota',	'CustomerID',	'Country',	'Amount']
sales_rd.drop(cols, axis=1, inplace=True)

In [95]:
sales_rd.head

<bound method NDFrame.head of       Unnamed: 0    BillDate  TotalAmount
0              0  14-11-2018        49.20
1              1  14-08-2019         7.30
2              2  10-05-2018        43.80
3              3  11-12-2017         4.55
4              4  28-01-2018        20.85
...          ...         ...          ...
1926        2463  04-11-2018        53.90
1927        2464  29-06-2018        25.65
1928        2465  05-12-2019        39.00
1929        2466  23-02-2019        21.90
1930        2468  01-07-2019         3.65

[1931 rows x 3 columns]>

In [96]:
sales_rd = sales_rd.sort_values('BillDate')

In [97]:
sales_rd.head

<bound method NDFrame.head of       Unnamed: 0    BillDate  TotalAmount
697          877  01-02-2018       34.500
598          750  01-02-2018        2.001
1409        1797  01-02-2018        3.250
436          550  01-02-2018       52.500
1475        1879  01-02-2018       24.600
...          ...         ...          ...
376          476  31-10-2019       98.400
1139        1444  31-10-2019       40.800
1580        2019  31-10-2019        5.290
712          896  31-10-2019       27.800
987         1254  31-10-2019      163.800

[1931 rows x 3 columns]>

In [98]:
sales_rd_02 = sales_rd.groupby('BillDate')['TotalAmount'].sum().reset_index()

In [99]:
sales_rd_02.head

<bound method NDFrame.head of        BillDate  TotalAmount
0    01-02-2018      116.851
1    01-02-2019     1029.220
2    01-03-2018        6.500
3    01-03-2019      584.840
4    01-04-2018      153.830
..          ...          ...
557  31-07-2019       50.620
558  31-08-2018       39.100
559  31-08-2019      119.250
560  31-10-2018       54.070
561  31-10-2019      393.920

[562 rows x 2 columns]>

In [100]:
#Seperating train and test from given data
sales_data = sales_rd_02['TotalAmount'].values
sales_data = sales_data.reshape((-1,1))

In [101]:
split_percent = 0.8
split = int(split_percent*len(sales_data))

In [102]:
sales_train = sales_data[:split]
sales_test = sales_data[split:]

In [103]:
date_train = sales_rd_02['BillDate'][:split]
date_test = sales_rd_02['BillDate'][split:]

In [104]:
print(len(sales_train))
print(len(sales_test))

449
113


In [105]:
look_back = 5 

In [106]:
train_generator = tf.keras.preprocessing.sequence.TimeseriesGenerator(sales_train, sales_train, length=look_back, batch_size=5)

In [107]:
test_generator = tf.keras.preprocessing.sequence.TimeseriesGenerator(sales_test, sales_test, length=look_back, batch_size=1)

In [108]:
model = Sequential()
model.add(
		LSTM(10, 
            activation= 'relu',
            input_shape= (look_back,1))
)

In [109]:
model.add(Dense(1))

In [110]:
model.compile(optimizer= 'adam', loss='mse')

In [111]:
num_epochs = 200 #test for 200 epochs

In [112]:
model.fit(train_generator, epochs=num_epochs, verbose=1)

Epoch 1/200
89/89 [==============================] - 8s 8ms/step - loss: 176539.0156
Epoch 2/200
89/89 [==============================] - 1s 8ms/step - loss: 173736.2188
Epoch 3/200
89/89 [==============================] - 1s 9ms/step - loss: 162808.0781
Epoch 4/200
89/89 [==============================] - 1s 10ms/step - loss: 158742.8594
Epoch 5/200
89/89 [==============================] - 1s 10ms/step - loss: 157150.9062
Epoch 6/200
89/89 [==============================] - 1s 8ms/step - loss: 155819.9531
Epoch 7/200
89/89 [==============================] - 1s 7ms/step - loss: 155481.8594
Epoch 8/200
89/89 [==============================] - 1s 7ms/step - loss: 156372.9844
Epoch 9/200
89/89 [==============================] - 1s 6ms/step - loss: 167617.6250
Epoch 10/200
89/89 [==============================] - 1s 8ms/step - loss: 159789.4219
Epoch 11/200
89/89 [==============================] - 1s 7ms/step - loss: 156687.4375
Epoch 12/200
89/89 [==============================] - 1s 7ms/

89/89 [==============================] - 1s 8ms/step - loss: 141741.8125
Epoch 97/200
89/89 [==============================] - 1s 7ms/step - loss: 141429.7188
Epoch 98/200
89/89 [==============================] - 1s 8ms/step - loss: 140794.1406
Epoch 99/200
89/89 [==============================] - 1s 13ms/step - loss: 142733.1562
Epoch 100/200
89/89 [==============================] - 1s 7ms/step - loss: 147157.0938
Epoch 101/200
89/89 [==============================] - 1s 11ms/step - loss: 145349.1094
Epoch 102/200
89/89 [==============================] - 1s 11ms/step - loss: 144886.2344
Epoch 103/200
89/89 [==============================] - 1s 9ms/step - loss: 141603.8281
Epoch 104/200
89/89 [==============================] - 1s 7ms/step - loss: 143068.9531
Epoch 105/200
89/89 [==============================] - 1s 7ms/step - loss: 143442.9688
Epoch 106/200
89/89 [==============================] - 1s 6ms/step - loss: 143791.2188
Epoch 107/200
89/89 [==============================] - 0s

89/89 [==============================] - 1s 6ms/step - loss: 131739.5469
Epoch 191/200
89/89 [==============================] - 1s 6ms/step - loss: 130801.0625
Epoch 192/200
89/89 [==============================] - 1s 6ms/step - loss: 129656.3047
Epoch 193/200
89/89 [==============================] - 1s 6ms/step - loss: 130905.6562
Epoch 194/200
89/89 [==============================] - 1s 6ms/step - loss: 132709.0312
Epoch 195/200
89/89 [==============================] - 1s 6ms/step - loss: 145275.0781
Epoch 196/200
89/89 [==============================] - 1s 6ms/step - loss: 144979.5625
Epoch 197/200
89/89 [==============================] - 1s 6ms/step - loss: 147951.1875
Epoch 198/200
89/89 [==============================] - 1s 6ms/step - loss: 144398.3906
Epoch 199/200
89/89 [==============================] - 1s 7ms/step - loss: 142645.7969
Epoch 200/200
89/89 [==============================] - 1s 8ms/step - loss: 141072.8906


In [114]:
prediction = model.predict(test_generator)

108/108 [==============================] - 0s 4ms/step


In [116]:
print(prediction)

[[ 67.790115]
 [ 75.62396 ]
 [217.11607 ]
 [ 83.51165 ]
 [ 96.96542 ]
 [157.10535 ]
 [148.41913 ]
 [156.80817 ]
 [185.98515 ]
 [167.08473 ]
 [118.231415]
 [ 69.5665  ]
 [ 94.48311 ]
 [106.31946 ]
 [369.1982  ]
 [246.24254 ]
 [114.14337 ]
 [ 94.879074]
 [ 72.55364 ]
 [272.17813 ]
 [144.20222 ]
 [169.30931 ]
 [126.4925  ]
 [ 56.41993 ]
 [128.97107 ]
 [146.36414 ]
 [189.82675 ]
 [226.19536 ]
 [282.63577 ]
 [223.2563  ]
 [169.52168 ]
 [ 64.79225 ]
 [ 34.21007 ]
 [ 42.024918]
 [147.89209 ]
 [101.439575]
 [105.54943 ]
 [121.817795]
 [120.86476 ]
 [ 83.06822 ]
 [111.98694 ]
 [142.4834  ]
 [120.91507 ]
 [142.38524 ]
 [179.70952 ]
 [360.56122 ]
 [188.50037 ]
 [163.31996 ]
 [237.34587 ]
 [ 65.540375]
 [ 97.04839 ]
 [108.58682 ]
 [ 57.95812 ]
 [ 88.01869 ]
 [177.91289 ]
 [ 73.95287 ]
 [138.07642 ]
 [107.11058 ]
 [139.65262 ]
 [179.23286 ]
 [214.46468 ]
 [115.08524 ]
 [ 81.12308 ]
 [106.33475 ]
 [ 77.161194]
 [ 64.38095 ]
 [ 77.32029 ]
 [ 94.42624 ]
 [106.88705 ]
 [102.07156 ]
 [ 88.98962 ]
 [ 84.

In [117]:
sales_train = sales_train.reshape((-1))
sales_test = sales_test.reshape((-1))
prediction = prediction.reshape((-1))

In [119]:
actual = sales_test[:116]

In [120]:
print(actual)

[ 196.1   115.49   74.01   42.2   185.9    44.1   259.37   15.99   79.5
   59.39   54.05   98.31  179.05   89.5    39.5    40.8    89.55   25.15
  122.44  147.65    9.78   17.5     5.75  307.18   53.79   18.25   44.55
   25.     23.85   29.7    54.75  109.1   231.92  529.87  105.09  171.95
  173.8     6.5    60.5   158.46   69.     65.7    60.1   197.57   48.6
   86.88    4.95   70.36   32.9   260.91  108.85  105.    227.34  206.25
   71.95  133.5    30.75  125.    128.35   39.6    48.53   89.64  126.13
  104.95  140.65  107.6   278.58  127.75  125.78   36.9   258.49  172.64
   60.5    37.03   94.15   50.14   66.2   824.5    39.5    27.3   930.1
  122.4    57.98  256.9    64.    227.7   279.41    6.5   135.8   372.41
   10.5    39.55   14.85 1926.2    53.75   47.7   141.72   42.75   68.4
 3014.54  290.29  328.55  137.    169.02  146.32   18.59   89.88   73.6
   50.62   39.1   119.25   54.07  393.92]


In [121]:
sales_data = sales_data.reshape((-1))  #numpy array

In [127]:
def predict(num_prediction, model):
    prediction_list = sales_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
    
    return prediction_list

In [132]:
def predict_dates(num_prediction):
        last_date = sales_rd_02['BillDate'].values[-1]
        prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
        return prediction_dates
    

1/1 [==============================] - 0s 146ms/step


In [135]:
forecasted_sales_data.reset_index()

,index,BillDate,PredictedAmount
0,0,2019-10-31,393.920000
1,1,2019-11-01,291.561127
2,2,2019-11-02,226.522385
3,3,2019-11-03,89.341324
4,4,2019-11-04,158.759293
5,5,2019-11-05,114.417313
6,6,2019-11-06,83.795395
7,7,2019-11-07,71.492096
8,8,2019-11-08,119.772522
9,9,2019-11-09,86.012726
